In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, NoSuchElementException

import pandas as pd
import requests
from bs4 import BeautifulSoup 
import time
import datetime
import smtplib 
import csv

In [2]:
# Load the Excel file
df=pd.read_excel(r"C:\Users\Administrator\Downloads\Book2.xlsx")
headers= {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"}
date=datetime.date.today()

In [3]:
def GetAthleteProfileByEra(athlete_name, era_value):
    # Set up the WebDriver
    service = Service("C:\\Users\\Administrator\\Downloads\\chromedriver-win64\\chromedriver.exe")
    driver = webdriver.Chrome(service=service)

    # Open Olympedia
    driver.get("https://www.olympedia.org/")

    # Find the search bar and input the athlete's name
    search_bar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "form-control"))
    )
    search_bar.send_keys(athlete_name)
    search_bar.send_keys(Keys.RETURN)
    # Check if the URL indicates a direct profile link
    current_url = driver.current_url
    #print("current URL after search:", current_url)
    if "quick_search" not in current_url:
        return current_url

    try:
        # Wait for the search results table to load
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//table[contains(@class, 'table-striped')]"))
        )

        # Retrieve all rows in the results table
        rows = driver.find_elements(By.XPATH, "//table[contains(@class, 'table-striped')]//tr")
        #print(f"Total rows found: {len(rows)}")  # Print the number of rows
        athlete_link = None

        # Check for the exact name match first
        for row in rows:
            try:
                # Print the HTML of the row for debugging
                row_html = row.get_attribute('outerHTML')
                #print("Row HTML:", row_html)

                # Extract name and era elements
                name_element = row.find_element(By.XPATH, ".//td[2]/a")  # Correct index for Name
                era_element = row.find_element(By.XPATH, ".//td[6]")  # Correct index for Era

                # Print the era and name for debugging
                #print("Era Element:", era_element.text)  # Print the era text
                #print("Name Element:", name_element.text)  # Print the name text

                # Compare the name and check the era
                if name_element.text.strip() in athlete_name:
                    # Check if the era contains the specified era_value
                    #print("the name is",name_element.text.strip())
                    if str(era_value) in era_element.text:
                        athlete_link = name_element.get_attribute('href')
                        break
            
            except NoSuchElementException:
                #print("Element not found in this row.")  # Print message if elements are not found
                continue  # Skip this row if the expected elements aren't found

        # If no link was found from the name match, check the era only
        if not athlete_link:
            for row in rows:
                try:
                    era_element = row.find_element(By.XPATH, ".//td[6]")  # Adjust index for Era
                    if str(era_value) in era_element.text:
                        name_element = row.find_element(By.XPATH, ".//td[2]/a")  # Adjust index for Name
                        athlete_link = name_element.get_attribute('href')
                        break
                except NoSuchElementException:
                    continue  # Skip this row if the expected elements aren't found

        return athlete_link if athlete_link else "Athlete not found."

    except TimeoutException:
        return "Timed out waiting for the search results."

    finally:
        # Close the browser
        driver.quit()


In [4]:
#profile_link = GetAthleteProfileByEra("Per Blom","1972")
#print(profile_link)


In [5]:
def GetAthletePhoto_OriginalName_BirthDate(profile_link):
    # Initialize variables to store extracted data
    athlete_image = None
    original_name = None
    born_info = None

    # Set up headers to mimic a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Request the page
    page = requests.get(profile_link, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    # Find the athlete's image link
    athlete_image_elem = soup.find(class_='photo')
    if athlete_image_elem:
        athlete_image = athlete_image_elem['src']  # Extract the 'src' attribute from the <img> tag
    
    # Helper function to find table row based on <th> text content (case insensitive)
    def find_row_by_header_text(soup, header_text):
        return soup.find('th', string=lambda x: x and header_text.lower() in x.lower())

    # Extract "Original Name"
    original_name_elem = find_row_by_header_text(soup, 'Original name')
    if original_name_elem:
        original_name = original_name_elem.find_next_sibling('td').get_text(strip=True)

    # Extract "Born" info
    born_elem = find_row_by_header_text(soup, 'Born')
    if born_elem:
        born_info = born_elem.find_next_sibling('td').get_text(strip=True)

    # Return the extracted values
    return athlete_image, original_name, born_info

In [12]:
results = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    athlete_name = row['Name']
    era_value = row['Year']
    
    # Get the athlete profile
    profile = GetAthleteProfileByEra(athlete_name, era_value)
    # Get the athlete Image and Born Date
    image_url = None
    original_name = None
    birthdate = None
    if 'http' in profile:
        [image_url, original_name, birthdate]=GetAthletePhoto_OriginalName_BirthDate(profile)
        
    # Store the result
    results.append({
        'ID': row['ID'],
        'Name': athlete_name,
        'Year': era_value,
        'Profile': profile,
        'Image':image_url,
        'Original Name': original_name,
        'Born' : birthdate
    })
    if index % 10 ==0:
        print("Ten Done")
    #if index == 10:
     #   break

# Convert results to a DataFrame for easier handling and output
results_df = pd.DataFrame(results)

# Save the results to a new Excel file
output_file_path = "C:\\Users\\Administrator\\Downloads\\AthleteProfiles.xlsx"
results_df.to_excel(output_file_path, index=False)

print(f"Athlete profiles saved to {output_file_path}.")

Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
Ten Done
T

ConnectionError: HTTPSConnectionPool(host='www.olympedia.org', port=443): Max retries exceeded with url: /athletes/37460 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001D5FB9FA890>: Failed to establish a new connection: [WinError 10065] A socket operation was attempted to an unreachable host'))

In [14]:
# Convert results to a DataFrame for easier handling and output
results_df = pd.DataFrame(results)

# Save the results to a new Excel file
output_file_path = "C:\\Users\\Administrator\\Downloads\\AthleteProfiles.xlsx"
results_df.to_excel(output_file_path, index=False)

print(f"Athlete profiles saved to {output_file_path}.")

Athlete profiles saved to C:\Users\Administrator\Downloads\AthleteProfiles.xlsx.
